This notebooks allows for the creation of sample ticker sets with category labels such as asset class, market capp, sector, etc.

Method 1: Trading View Screens Export

In [1]:
import os
import pandas as pd
import random

In [2]:
# For US Stock Data
screen_date = "02_24_25"
market_caps = ["MICROCAP", "SMALLCAP", "MIDCAP", "LARGECAP", "MEGACAP"]

# Read all files in TradingView Screens folder with specified filters
# Define the folder path
folder_path = "TV_Screens"

# Create an empty list to store dataframes
dfs = []

# Loop through each market cap and read the corresponding files
for market_cap in market_caps:
    file_pattern = f"TV_SCREEN_US_STOCK_{market_cap}_{screen_date}.csv"
    file_path = os.path.join(os.getcwd(), folder_path, file_pattern)
    df = pd.read_csv(file_path)
    df['Market cap tag'] = market_cap
    dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
sample_ticker_list = []
# For each market cap
for market_cap in market_caps:
    # Filter the data
    temp_list = []
    filtered_df = combined_df[combi ned_df['Market cap tag'] == market_cap]
    # For each sector in the filtered data
    for sector in filtered_df['Sector'].unique():
        # Filter the data
        sector_df = filtered_df[filtered_df['Sector'] == sector]
        # Sample upto 2 tickers from the filtered data or all if less than 2
        for ticker in sector_df.sample(n=min(2, len(sector_df)))['Symbol']:
            temp_list.append({"ticker": ticker, "sector": sector, "market_cap": market_cap})

    sample_ticker_list.append(random.sample(temp_list, min(2, len(temp_list))))

flattened_ticker_list = [ticker for sublist in sample_ticker_list for ticker in sublist]

In [4]:
len(flattened_ticker_list)

10

In [5]:
import nest_asyncio
nest_asyncio.apply()

import os
import sys

sys.path.append(os.path.join(os.getcwd(), "../.."))

from jarjarquant import Jarjarquant
jjq = Jarjarquant()

In [6]:
# Use the get_tws_ticker method from data gatherer
# Empty end date implies fetching data upto the last available date
# df = jjq.data_gatherer.async_get_tws_ticker(ticker=ticker, exchange='SMART', currency='USD', end_date='', duration='1 D', bar_size='1 min', what_to_show='TRADES')
bar_size = '1 day'

In [7]:
results = jjq.data_gatherer.get_random_price_samples_tws(years_in_sample=10, tickers=[flattened_ticker_list[i]['ticker'] for i in range(len(flattened_ticker_list))], num_tickers_to_sample=10, verbose=True)

In [12]:
len(results)

10

In [13]:
flat_results = {key: value for result in results for key, value in result.items()}

In [10]:
ticker_data = result = {item['ticker']: {'market_cap': item['market_cap'], 'sector': item['sector']} for item in flattened_ticker_list}

In [ ]:
from datetime import datetime

today_date = datetime.today().strftime('%Y-%m-%d')
for key, value in flat_results.items():
    value.to_csv(f"data/sample_daily_10_all/{ticker_data[key]['market_cap']}_{ticker_data[key]['sector']}_{bar_size}_{today_date}.csv")